In [176]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('./metrics_freedrag_experiments_1.csv')

In [3]:
df.head()

,Category,Metric,Value,Eval_Root
0,art_work,avg LPIPS,0.957014,../freedrag_experiments/freedrag_diffusion_res...
1,land_scape,avg LPIPS,0.943471,../freedrag_experiments/freedrag_diffusion_res...
2,building_city_view,avg LPIPS,0.941149,../freedrag_experiments/freedrag_diffusion_res...
3,building_countryside_view,avg LPIPS,0.941073,../freedrag_experiments/freedrag_diffusion_res...
4,animals,avg LPIPS,0.968489,../freedrag_experiments/freedrag_diffusion_res...


In [7]:
df['Eval_Root'].unique().shape

(128,)

In [9]:
4*4*8

128

In [125]:
len([exp_name for exp_name in df['Eval_Root'].unique() if df[df['Eval_Root'] == exp_name].shape[0] != 20])

21

In [122]:
for exp_name in df['Eval_Root'].unique():4*8
    exp = df[df['Eval_Root'] == exp_name]
    if exp.shape[0] != 20:
        print(exp_name)
    

../freedrag_experiments/freedrag_diffusion_res_80_0.7_0.01_3_reduce_dims=(0-1)_n_step=300_d_max=3.0_l_expected=0.3_L1m=True_L1p=False_L1mask=True
../freedrag_experiments/freedrag_diffusion_res_80_0.7_0.01_3_reduce_dims=(0-1)_n_step=300_d_max=3.0_l_expected=0.3_L1m=True_L1p=True_L1mask=True
../freedrag_experiments/freedrag_diffusion_res_80_0.7_0.01_3_reduce_dims=(0-1)_n_step=300_d_max=3.0_l_expected=0.3_L1m=False_L1p=True_L1mask=True
../freedrag_experiments/freedrag_diffusion_res_80_0.7_0.01_3_reduce_dims=(0-1)_n_step=300_d_max=3.0_l_expected=0.3_L1m=False_L1p=True_L1mask=False
../freedrag_experiments/freedrag_diffusion_res_80_0.7_0.01_3_reduce_dims=(0-1)_n_step=300_d_max=3.0_l_expected=0.3_L1m=False_L1p=False_L1mask=True
../freedrag_experiments/freedrag_diffusion_res_80_0.7_0.01_3_reduce_dims=(0-1)_n_step=300_d_max=4.5_l_expected=0.45_L1m=False_L1p=True_L1mask=True
../freedrag_experiments/freedrag_diffusion_res_80_0.7_0.01_3_reduce_dims=(0-1)_n_step=300_d_max=4.5_l_expected=0.45_L1m=Tr

In [131]:
def extract_experiment_values(experiment_string):
    # Dictionary to store our patterns and their corresponding values
    patterns = {
        'reduce_dims': r'reduce_dims=\((.*?)\)',  # Special case for tuple
        'd_max': r'd_max=(\d+\.?\d*)',
        'l_expected': r'l_expected=(\d+\.?\d*)',
        'L1m': r'L1m=(True|False)',
        'L1p': r'L1p=(True|False)',
        'L1mask': r'L1mask=(True|False)'
    }
    
    # Dictionary to store our results
    results = {}
    
    # Extract each value
    for key, pattern in patterns.items():
        match = re.search(pattern, experiment_string)
        if match:
            value = match.group(1)
            # Convert string to appropriate type
            if key == 'reduce_dims':
                
                value = tuple(map(int, value.replace('-', ',').split(','))) if value else tuple()
            elif key in ['L1m', 'L1p', 'L1mask']:
                value = value == 'True'
            else:
                value = float(value)
            results[key] = value
        else:
            results[key] = 'all'
    
    return pd.Series(results)

extracted_columns = df['Eval_Root'].apply(extract_experiment_values)
df_expanded = pd.concat([df, extracted_columns], axis=1)
df_expanded['d_l_params'] = list(zip(df_expanded['d_max'], df_expanded['l_expected']))
df_expanded = df_expanded.drop(['Eval_Root', 'd_max', 'l_expected'], axis=1)

df_expanded

,Category,Metric,Value,reduce_dims,L1m,L1p,L1mask,d_l_params
0,art_work,avg LPIPS,0.957014,"(0,)",False,True,True,"(1.5, 0.15)"
1,land_scape,avg LPIPS,0.943471,"(0,)",False,True,True,"(1.5, 0.15)"
2,building_city_view,avg LPIPS,0.941149,"(0,)",False,True,True,"(1.5, 0.15)"
3,building_countryside_view,avg LPIPS,0.941073,"(0,)",False,True,True,"(1.5, 0.15)"
4,animals,avg LPIPS,0.968489,"(0,)",False,True,True,"(1.5, 0.15)"
...,...,...,...,...,...,...,...,...
2371,human_head,avg MD,71.830905,"(0,)",False,True,True,"(5.0, 1.0)"
2372,human_upper_body,avg MD,71.384134,"(0,)",False,True,True,"(5.0, 1.0)"
2373,human_full_body,avg MD,37.772799,"(0,)",False,True,True,"(5.0, 1.0)"
2374,interior_design,avg MD,42.387486,"(0,)",False,True,True,"(5.0, 1.0)"


In [262]:
def compute_mean_if_valid(group):
    if group["Category"].nunique() == 10:
        
        return group.select_dtypes(include=[np.number]).apply(lambda col: col.mean() if col.notna().all() else np.nan)
    
    else:
        # display(group.select_dtypes(include=[np.number]))
        # display(group.select_dtypes(include=[np.number])[group.select_dtypes(include=[np.number]).isna().any(axis=0)])
        return pd.Series(np.nan, index=group.select_dtypes(include=[np.number]).columns)

In [263]:
df_expanded_pivot_LPIPS = df_expanded[df_expanded["Metric"] == "avg LPIPS"].pivot_table(
    index=["reduce_dims", "d_l_params", "Category"], 
    values="Value",             
    columns=["L1m", "L1p", "L1mask"]        
).reset_index()

df_expanded_pivot_MD = df_expanded[df_expanded["Metric"] == "avg MD"].pivot_table(
    index=["reduce_dims", "d_l_params", "Category"], 
    values="Value",             
    columns=["L1m", "L1p", "L1mask"]        
).reset_index()

In [264]:
# df_expanded_pivot_LPIPS[]

# df_expanded_pivot_LPIPS[df_expanded_pivot_LPIPS.isna()]

nan_columns = df_expanded_pivot_LPIPS.columns[df_expanded_pivot_LPIPS.isna().any()].tolist()
print(nan_columns)

[(False, False, True), (False, True, False), (False, True, True), (True, False, True), (True, True, True)]


In [321]:
LPIPS_mean_df = df_expanded_pivot_LPIPS.groupby(["reduce_dims", "d_l_params"]).apply(compute_mean_if_valid).reset_index()

MD_mean_df = df_expanded_pivot_MD.groupby(["reduce_dims", "d_l_params"]).apply(compute_mean_if_valid).reset_index()

/tmp/ipykernel_3022170/2460301679.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  LPIPS_mean_df = df_expanded_pivot_LPIPS.groupby(["reduce_dims", "d_l_params"]).apply(compute_mean_if_valid).reset_index()
/tmp/ipykernel_3022170/2460301679.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  LPIPS_mean_df = df_expanded_pivot_LPIPS.groupby(["reduce_dims", "d_l_params"]).apply(compute_mean_if_valid).reset_index()
/tmp/ipykernel_3022170/2460301679.py:3: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  MD_mean_df = df_expanded_pivot_MD.groupby(["reduce_dims", "d_l_params"]

In [383]:
def reorder_columns(df):

    df_copy = df.copy(deep=True)
    df_sorted = df_copy.swaplevel(0, 2, axis=1)
    
    new_columns = list(df_sorted.columns)
    new_columns[2:] = sorted(new_columns[2:], key=lambda x: (x[0], x[1], x[2]), reverse=True)

    return df_sorted.reindex(new_columns, axis=1)


In [384]:
LPIPS_mean_df_sorted = reorder_columns(LPIPS_mean_df)
LPIPS_mean_df_sorted

L1mask                               True                                \
L1p                                  True               False             
L1m    reduce_dims   d_l_params      True     False      True     False   
0           (0, 1)  (1.5, 0.15)  0.960686  0.962787  0.961062  0.962751   
1           (0, 1)   (3.0, 0.3)       NaN       NaN       NaN       NaN   
2           (0, 1)  (4.5, 0.45)       NaN       NaN       NaN       NaN   
3           (0, 1)   (5.0, 1.0)       NaN       NaN       NaN       NaN   
4             (0,)  (1.5, 0.15)  0.963553  0.963918  0.963673  0.963902   
5             (0,)   (3.0, 0.3)  0.963282  0.963400  0.963490  0.963293   
6             (0,)  (4.5, 0.45)  0.963185  0.962366  0.963464  0.962166   
7             (0,)   (5.0, 1.0)  0.963184  0.962882  0.963309  0.962739   
8             (1,)  (1.5, 0.15)  0.963265  0.962150  0.962908  0.962791   
9             (1,)   (3.0, 0.3)  0.962156  0.961533  0.962142  0.962026   
10            (1,)  (4.5, 0.45)  0.961872  0.960878  0.961698  0.961731   
11            (1,)   (5.0, 1.0)  0.961687  0.960640  0.962259  0.961773   
12             all  (1.5, 0.15)  0.960019  0.908737  0.960919  0.937307   
13             all   (3.0, 0.3)  0.950043  0.915833  0.951196  0.922300   
14             all  (4.5, 0.45)  0.934537  0.910175  0.935187  0.912857   
15             all   (5.0, 1.0)  0.900773  0.886664  0.904722  0.890596   

L1mask     False                                
L1p         True               False            
L1m         True     False      True     False  
0       0.961109  0.962705  0.960540  0.962672  
1       0.960524       NaN  0.960873  0.963099  
2            NaN       NaN       NaN       NaN  
3            NaN       NaN       NaN       NaN  
4       0.962994  0.963891  0.962969  0.963817  
5       0.962636  0.963295  0.962723  0.963186  
6       0.962320  0.962205  0.962568  0.962097  
7       0.962149  0.962784  0.962277  0.962677  
8       0.962152  0.962057  0.961826  0.962796  
9       0.960947  0.961200  0.960934  0.961839  
10      0.960349  0.960978  0.960713  0.961556  
11      0.960147  0.960648  0.960836  0.961653  
12      0.951883  0.871411  0.954353  0.910883  
13      0.930120  0.879762  0.933350  0.886255  
14      0.905371  0.868952  0.905728  0.872503  
15      0.854323  0.844078  0.860537  0.849919

In [385]:
print(LPIPS_mean_df_sorted.fillna('$*$').to_latex(index=False, float_format="%.4f"))

\begin{tabular}{llllllllll}
\toprule
\multicolumn{2}{r}{} & \multicolumn{4}{r}{True} & \multicolumn{4}{r}{False} \\
\multicolumn{2}{r}{} & \multicolumn{2}{r}{True} & \multicolumn{2}{r}{False} & \multicolumn{2}{r}{True} & \multicolumn{2}{r}{False} \\
reduce_dims & d_l_params & True & False & True & False & True & False & True & False \\
\midrule
(0, 1) & (1.5, 0.15) & 0.9607 & 0.9628 & 0.9611 & 0.9628 & 0.9611 & 0.9627 & 0.9605 & 0.9627 \\
(0, 1) & (3.0, 0.3) & $*$ & $*$ & $*$ & $*$ & 0.9605 & $*$ & 0.9609 & 0.9631 \\
(0, 1) & (4.5, 0.45) & $*$ & $*$ & $*$ & $*$ & $*$ & $*$ & $*$ & $*$ \\
(0, 1) & (5.0, 1.0) & $*$ & $*$ & $*$ & $*$ & $*$ & $*$ & $*$ & $*$ \\
(0,) & (1.5, 0.15) & 0.9636 & 0.9639 & 0.9637 & 0.9639 & 0.9630 & 0.9639 & 0.9630 & 0.9638 \\
(0,) & (3.0, 0.3) & 0.9633 & 0.9634 & 0.9635 & 0.9633 & 0.9626 & 0.9633 & 0.9627 & 0.9632 \\
(0,) & (4.5, 0.45) & 0.9632 & 0.9624 & 0.9635 & 0.9622 & 0.9623 & 0.9622 & 0.9626 & 0.9621 \\
(0,) & (5.0, 1.0) & 0.9632 & 0.9629 & 0.9633 & 0.9627

In [386]:
MD_mean_df

L1m    reduce_dims   d_l_params      False                                   \
L1p                                  False                  True              
L1mask                               False       True      False       True   
0           (0, 1)  (1.5, 0.15)  53.751978  53.809118  53.696336  53.776436   
1           (0, 1)   (3.0, 0.3)  53.670384        NaN        NaN        NaN   
2           (0, 1)  (4.5, 0.45)        NaN        NaN        NaN        NaN   
3           (0, 1)   (5.0, 1.0)        NaN        NaN        NaN        NaN   
4             (0,)  (1.5, 0.15)  53.803123  53.745610  53.652153  53.715127   
5             (0,)   (3.0, 0.3)  53.734641  53.733747  53.848399  53.717426   
6             (0,)  (4.5, 0.45)  53.904659  53.813634  53.874530  53.845749   
7             (0,)   (5.0, 1.0)  53.781399  53.821017  53.723435  53.654424   
8             (1,)  (1.5, 0.15)  53.673373  53.729870  53.726352  53.655786   
9             (1,)   (3.0, 0.3)  53.759679  53.771972  53.620246  53.757455   
10            (1,)  (4.5, 0.45)  53.715657  53.768723  53.874980  53.710379   
11            (1,)   (5.0, 1.0)  53.672178  53.834622  53.691953  53.739176   
12             all  (1.5, 0.15)  43.001930  42.317988  35.141591  35.744569   
13             all   (3.0, 0.3)  38.286350  37.863567  36.706461  35.697713   
14             all  (4.5, 0.45)  35.684442  35.135168  35.224816  34.780528   
15             all   (5.0, 1.0)  29.790408  31.532760  29.280526  30.182484   

L1m          True                                   
L1p         False                  True             
L1mask      False       True      False       True  
0       53.922220  53.870870  53.954039  54.054011  
1       53.887506        NaN  53.943193        NaN  
2             NaN        NaN        NaN        NaN  
3             NaN        NaN        NaN        NaN  
4       53.508054  53.673167  53.526053  53.579714  
5       53.456089  53.637234  53.589742  53.504674  
6       53.615955  53.557071  53.641132  53.512124  
7       53.668238  53.537942  53.693627  53.582809  
8       53.720863  53.748523  53.678759  53.652477  
9       53.789888  53.741978  53.759531  53.786062  
10      53.966684  53.845484  54.092555  53.962431  
11      53.822743  53.955826  53.982071  53.864429  
12      52.794353  52.774192  52.538871  52.272005  
13      48.846406  48.601901  48.255609  47.821368  
14      40.705096  41.567164  40.872507  41.839550  
15      31.094077  34.044290  29.393875  33.340502

In [387]:
MD_mean_df_sorted = reorder_columns(MD_mean_df)
MD_mean_df_sorted

L1mask                                True                                   \
L1p                                   True                 False              
L1m    reduce_dims   d_l_params       True      False       True      False   
0           (0, 1)  (1.5, 0.15)  54.054011  53.776436  53.870870  53.809118   
1           (0, 1)   (3.0, 0.3)        NaN        NaN        NaN        NaN   
2           (0, 1)  (4.5, 0.45)        NaN        NaN        NaN        NaN   
3           (0, 1)   (5.0, 1.0)        NaN        NaN        NaN        NaN   
4             (0,)  (1.5, 0.15)  53.579714  53.715127  53.673167  53.745610   
5             (0,)   (3.0, 0.3)  53.504674  53.717426  53.637234  53.733747   
6             (0,)  (4.5, 0.45)  53.512124  53.845749  53.557071  53.813634   
7             (0,)   (5.0, 1.0)  53.582809  53.654424  53.537942  53.821017   
8             (1,)  (1.5, 0.15)  53.652477  53.655786  53.748523  53.729870   
9             (1,)   (3.0, 0.3)  53.786062  53.757455  53.741978  53.771972   
10            (1,)  (4.5, 0.45)  53.962431  53.710379  53.845484  53.768723   
11            (1,)   (5.0, 1.0)  53.864429  53.739176  53.955826  53.834622   
12             all  (1.5, 0.15)  52.272005  35.744569  52.774192  42.317988   
13             all   (3.0, 0.3)  47.821368  35.697713  48.601901  37.863567   
14             all  (4.5, 0.45)  41.839550  34.780528  41.567164  35.135168   
15             all   (5.0, 1.0)  33.340502  30.182484  34.044290  31.532760   

L1mask      False                                   
L1p          True                 False             
L1m          True      False       True      False  
0       53.954039  53.696336  53.922220  53.751978  
1       53.943193        NaN  53.887506  53.670384  
2             NaN        NaN        NaN        NaN  
3             NaN        NaN        NaN        NaN  
4       53.526053  53.652153  53.508054  53.803123  
5       53.589742  53.848399  53.456089  53.734641  
6       53.641132  53.874530  53.615955  53.904659  
7       53.693627  53.723435  53.668238  53.781399  
8       53.678759  53.726352  53.720863  53.673373  
9       53.759531  53.620246  53.789888  53.759679  
10      54.092555  53.874980  53.966684  53.715657  
11      53.982071  53.691953  53.822743  53.672178  
12      52.538871  35.141591  52.794353  43.001930  
13      48.255609  36.706461  48.846406  38.286350  
14      40.872507  35.224816  40.705096  35.684442  
15      29.393875  29.280526  31.094077  29.790408

In [388]:
print(MD_mean_df_sorted.fillna('$*$').to_latex(index=False, float_format="%.4f"))

\begin{tabular}{llllllllll}
\toprule
\multicolumn{2}{r}{} & \multicolumn{4}{r}{True} & \multicolumn{4}{r}{False} \\
\multicolumn{2}{r}{} & \multicolumn{2}{r}{True} & \multicolumn{2}{r}{False} & \multicolumn{2}{r}{True} & \multicolumn{2}{r}{False} \\
reduce_dims & d_l_params & True & False & True & False & True & False & True & False \\
\midrule
(0, 1) & (1.5, 0.15) & 54.0540 & 53.7764 & 53.8709 & 53.8091 & 53.9540 & 53.6963 & 53.9222 & 53.7520 \\
(0, 1) & (3.0, 0.3) & $*$ & $*$ & $*$ & $*$ & 53.9432 & $*$ & 53.8875 & 53.6704 \\
(0, 1) & (4.5, 0.45) & $*$ & $*$ & $*$ & $*$ & $*$ & $*$ & $*$ & $*$ \\
(0, 1) & (5.0, 1.0) & $*$ & $*$ & $*$ & $*$ & $*$ & $*$ & $*$ & $*$ \\
(0,) & (1.5, 0.15) & 53.5797 & 53.7151 & 53.6732 & 53.7456 & 53.5261 & 53.6522 & 53.5081 & 53.8031 \\
(0,) & (3.0, 0.3) & 53.5047 & 53.7174 & 53.6372 & 53.7337 & 53.5897 & 53.8484 & 53.4561 & 53.7346 \\
(0,) & (4.5, 0.45) & 53.5121 & 53.8457 & 53.5571 & 53.8136 & 53.6411 & 53.8745 & 53.6160 & 53.9047 \\
(0,) & (5.0, 1.0) 